In [ ]:
def ensemble_model(pythonData):
    """
    Function to run all models, and return the one with lowest RMSE.
    
    """

    # dictionary to save predictions for each point
    scores_df_dict = {
    "pointID": [],
    "predictions": [],
    "rmse": [],
    "modelName": []
    }

    # Create a DataFrame from the dictionary
    scores_df = pd.DataFrame(scores_df_dict)

    for i, row in pythonData.iterrows():
        pointID = row["pointID"]
        df = row["his"].to_dataframe()
        df.set_index(df.columns[0], inplace=True, drop=True)
        length_of_missing_data = row["dqDuration"]
        data_logging_interval = row["pointInterval"]

        # number of predictions
        horizon = int(length_of_missing_data/data_logging_interval)

        # training set size (relative to the horizon/prediction size)
        training_set_size = horizon * 10
        #training_set_size = int(228)                                         ################## REMOVE THIS
        
        # training / testing set to evaluate the model (relative to horizon of prediction)
        train_data = df.iloc[-1*int(training_set_size):]#.reset_index(drop=True)
        test_data = df.iloc[:-1*int(training_set_size)]#.reset_index(drop=True)


        seasonal_naive_predictions = seasonal_naive(train_data, length_of_missing_data, data_logging_interval)
        model_name = "Seasonal Naive"
        
        rmse_score = mean_squared_error(test_data[test_data.columns[0]].to_numpy(), seasonalNaivePredictions[seasonalNaivePredictions.columns[0]].to_numpy(), squared=False)

        # append data to the scored DF
        row_to_append = {'pointID': pointID, 'predictions': seasonal_naive_predictions, "rmse": "", "modelName": model_name}
        scores_df = pd.concat([scores_df, pd.DataFrame([row_to_append])], ignore_index=True)

        
    return scores_df